In [1]:
import psycopg2

In [2]:
from constants import PGDB_PASS
from vector_management import get_MiniLM_embedding

c:\Users\lenovo\anaconda3\envs\pinecone\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
conn = psycopg2.connect(
            dbname='knowledge_base',
            user='postgres',
            password=PGDB_PASS,
            host='localhost',
            port=5432
        )
cur = conn.cursor()

In [4]:
query_embedding = get_MiniLM_embedding("What's the Bonaparte Paper?")

In [5]:
 # See paper titles in database
cur.execute(f"SELECT * FROM paper_titles;")
query_results = cur.fetchall()

In [6]:
for result in query_results:
    print(result)

('8c941b1518412bff0d533886d431b90fa0fb36f51275d2f0d145ab17cb94e740', 'Automated Information Retrieval', ['N.M Hernandez', 'P.J. Lucafias', 'J.C. Graciosa'], 'C:\\Users\\lenovo\\Desktop\\OJTChatbotBEIC\\test_papers\\Automated_Info_Retrieval.txt', ['Machine Learning', 'Iraya', 'Documents'])
('d1df5e57a8004a06ef4324a45d48d9574e00034425a7ce3397ce769a4937cc11', 'A Case Study of Understanding the Bonaparte Basin', ['A.N.Sazali', 'N.M. Hernandez', 'F. Baillard', 'K.G. Maver'], 'C:\\Users\\lenovo\\Desktop\\OJTChatbotBEIC\\test_papers\\Bonaparte_Paper.txt', ['Case Study', 'Machine Learning', 'Iraya', 'Documents', 'Geology'])
('f8f1e133b9e1080b74252ffc72299d997aa3c0520d051d37e6d9b05ef62f5eb6', 'Sustainable Data Mining', ['T. Looi', 'N.E. Arif', 'N.M. Hernandez', 'F. Baillard'], 'C:\\Users\\lenovo\\Desktop\\OJTChatbotBEIC\\test_papers\\Sustainable_data_mining.txt', ['Data Mining', 'Oil', 'Gas', 'Iraya', 'Documents'])


In [7]:
 # Use L1 distance to query vectors 
cur.execute(f"""
            SELECT * FROM chunks 
            ORDER BY embedding <+> '{query_embedding.tolist()}' 
            LIMIT 5;
            """)
query_results = cur.fetchall()

In [8]:
for i, query_result in enumerate(query_results):
    print(f"RESULT {i+1}: \n{query_result[1].strip()}\n")

RESULT 1: 
The objective of this study is to
understand and obtain meaningful insights into the Bonaparte Basin based on the substantial amount
of information available in previous studies, reports and presentations

RESULT 2: 
Methodology

As of 2021, the Bonaparte Basin encompasses 440 wells representing 58 years of exploration history
summarized in over 270,000 pages of documents and in 250,000 images

RESULT 3: 
The heatmaps show the distribution of clastic and carbonates
across the Bonaparte Basin and identify patterns and anomalies present in the area

RESULT 4: 
In this case study the area of interest covers Bonaparte Basin, which is located north-west of
the Australian continental margin (Figure 1)

RESULT 5: 
Based on a quick look and gathering of all information it can be concluded that most of the
production in the Bonaparte Basin is from Jurassic and Triassic with observed net pay ~18-60m
thickness, porosity ~11-29% and saturation ~11-55% Sw



In [9]:
cur.close()
conn.close()